In [1]:
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 14, 'font.family': 'serif'})

In [2]:
# plot runtime in n, for m=2

xticks = np.arange(10,51,10)
df = pd.concat([pd.read_csv("../figs/runtime_in_n" + str(n) + "_m2.csv") for n in xticks])

x = df['n']
cputime = df['cputime']
cputime_std_error = df['cputime_std_error']
iterations = df['iterations']
iterations_std_error = df['iterations_std_error']

xlim = (8,51)

fig = plt.figure(figsize=(10, 5))

ax1 = fig.add_subplot(2, 3, 1)
ax1.plot(x, cputime, color='black')
ax1.plot(x, cputime + 1.96 * cputime_std_error, color='grey', linestyle=':')
ax1.plot(x, cputime - 1.96 * cputime_std_error, color='grey', linestyle=':')
ax1.set_xticks(xticks)
ax1.set_xlim(xlim)
ax1.set_title("m=2")
ax1.set_yticks([0,0.2,0.4,0.6])
ax1.set_ylim((-0.05,0.7))
ax1.set_ylabel('CPU time (s)')

for label in ax1.get_xticklabels():
    label.set_color("white")
    label.set_fontsize(1)

ax2 = fig.add_subplot(2, 3, 4)
ax2.plot(x, iterations, color='black')
ax2.plot(x, iterations + 1.96 * iterations_std_error, color='grey', linestyle=':')
ax2.plot(x, iterations - 1.96 * iterations_std_error, color='grey', linestyle=':')
ax2.set_xticks(xticks)
ax2.set_xlim(xlim)
ax2.set_yticks([5,6,7,8])
ax2.set_ylim((4.5,8.5))
ax2.set_ylabel('# iterations')
ax2.set_xlabel('n')

# plot runtime in n, for m=3

df = pd.concat([pd.read_csv("../figs/runtime_in_n" + str(n) + "_m3.csv") for n in xticks])
x = df['n']
cputime = df['cputime']
cputime_std_error = df['cputime_std_error']
iterations = df['iterations']
iterations_std_error = df['iterations_std_error']

ax3 = fig.add_subplot(2, 3, 2)
ax3.set_title("m=3")
ax3.plot(x, cputime, color='black')
ax3.plot(x, cputime + 1.96 * cputime_std_error, color='grey', linestyle=':')
ax3.plot(x, cputime - 1.96 * cputime_std_error, color='grey', linestyle=':')
ax3.set_xticks(xticks)
ax3.set_xlim(xlim)
ax3.set_yticks([0,2,4,6])
ax3.set_ylim((-0.5,7))

for label in ax3.get_xticklabels():
    label.set_color("white")
    label.set_fontsize(1)

ax4 = fig.add_subplot(2, 3, 5)
ax4.plot(x, iterations, color='black')
ax4.plot(x, iterations + 1.96 * iterations_std_error, color='grey', linestyle=':')
ax4.plot(x, iterations - 1.96 * iterations_std_error, color='grey', linestyle=':')
ax4.set_xticks(xticks)
ax4.set_xlim(xlim)
ax4.set_yticks([5,6,7,8])
ax4.set_ylim((4.5,8.5))
ax4.set_xlabel('n')

# plot runtime in n, for m=4

df = pd.concat([pd.read_csv("../figs/runtime_in_n" + str(n) + "_m4.csv") for n in xticks])
x = df['n']
cputime = df['cputime']
cputime_std_error = df['cputime_std_error']
iterations = df['iterations']
iterations_std_error = df['iterations_std_error']

ax5 = fig.add_subplot(2, 3, 3)
ax5.set_title("m=4")
ax5.plot(x, cputime, color='black')
ax5.plot(x, cputime + 1.96 * cputime_std_error, color='grey', linestyle=':')
ax5.plot(x, cputime - 1.96 * cputime_std_error, color='grey', linestyle=':')
ax5.set_xticks(xticks)
ax5.set_xlim(xlim)
ax5.set_yticks([0,100,200,300])
ax5.set_ylim((-50,350))

for label in ax5.get_xticklabels():
    label.set_color("white")
    label.set_fontsize(1)

ax6 = fig.add_subplot(2, 3, 6)
ax6.plot(x, iterations, color='black')
ax6.plot(x, iterations + 1.96 * iterations_std_error, color='grey', linestyle=':')
ax6.plot(x, iterations - 1.96 * iterations_std_error, color='grey', linestyle=':')
ax6.set_xticks(xticks)
ax6.set_xlim(xlim)
ax6.set_yticks([5,6,7,8])
ax6.set_ylim((4.5,8.5))
ax6.set_xlabel('n')

plt.tight_layout()
plt.savefig("../figs/runtime_in_n.pdf")
plt.show()

FileNotFoundError: [Errno 2] File b'../figs/runtime_in_n10_m2.csv' does not exist: b'../figs/runtime_in_n10_m2.csv'

In [ ]:
# plot runtime in m for n fixed

xticks = [1,2,3,4]

n = 10
df = pd.concat([pd.read_csv(f"figs/runtime_in_m{m}_n{n}.csv") for m in xticks])
cputime = df['cputime']
cputime_std_error = df['cputime_std_error']
iterations = df['iterations']
iterations_std_error = df['iterations_std_error']

fig = plt.figure(figsize=(10,5))

ax1 = fig.add_subplot(2, 3, 1)
ax1.semilogy(xticks, cputime, color='black')
ax1.semilogy(xticks, cputime + 1.96 * cputime_std_error, color='grey', linestyle=':')
ax1.semilogy(xticks, cputime - 1.96 * cputime_std_error, color='grey', linestyle=':')
# ax1.set_yticks([1e02, 1e01, 1e00, 1e01, 1e02, 1e03, 1e04])
# ax1.set_ylim((0.5190408372055083, 267.28485384033854))
ax1.set_title("n="+str(n))
ax1.set_ylabel('CPU time (s)')

for label in ax1.get_xticklabels():
    label.set_color("white")
    label.set_fontsize(0)

ax2 = fig.add_subplot(2, 3, 4)

ax2.plot(xticks, iterations, color='black')
ax2.plot(xticks, iterations + 1.96 * iterations_std_error, color='grey', linestyle=':')
ax2.plot(xticks, iterations - 1.96 * iterations_std_error, color='grey', linestyle=':')

ax2.set_xticks([1,2,3,4])
ax2.set_xlim((0.8,4.2))
ax2.set_yticks([5,6,7,8])
ax2.set_ylim((4.5,8.5))
ax2.set_ylabel('# iterations')
ax2.set_xlabel('m')

n = 30

df = pd.concat([pd.read_csv(f"figs/runtime_in_m{m}_n{n}.csv") for m in xticks])
cputime = df['cputime']
cputime_std_error = df['cputime_std_error']
iterations = df['iterations']
iterations_std_error = df['iterations_std_error']

ax3 = fig.add_subplot(2, 3, 2, sharey=ax1)
ax3.semilogy(xticks, cputime, color='black')
ax3.semilogy(xticks, cputime + 1.96 * cputime_std_error, color='grey', linestyle=':')
ax3.semilogy(xticks, cputime - 1.96 * cputime_std_error, color='grey', linestyle=':')
ax3.set_title("n=" + str(n))

for label in ax3.get_yticklabels():
    label.set_color("white")
    label.set_fontsize(0)

for label in ax3.get_xticklabels():
    label.set_color("white")
    label.set_fontsize(0)

ax4 = fig.add_subplot(2, 3, 5, sharey=ax2)
ax4.plot(xticks, iterations, color='black')
ax4.plot(xticks, iterations + 1.96 * iterations_std_error, color='grey', linestyle=':')
ax4.plot(xticks, iterations - 1.96 * iterations_std_error, color='grey', linestyle=':')
ax4.set_xlabel('m')

for label in ax4.get_yticklabels():
    label.set_color("white")
    label.set_fontsize(0)
    
n = 50

df = pd.concat([pd.read_csv(f"figs/runtime_in_m{m}_n{n}.csv") for m in xticks])
cputime = df['cputime']
cputime_std_error = df['cputime_std_error']
iterations = df['iterations']
iterations_std_error = df['iterations_std_error']

ax5 = fig.add_subplot(2, 3, 6, sharey=ax2)
ax5.plot(xticks, iterations, color='black')
ax5.plot(xticks, iterations + 1.96 * iterations_std_error, color='grey', linestyle=':')
ax5.plot(xticks, iterations - 1.96 * iterations_std_error, color='grey', linestyle=':')
ax5.set_xlabel('m')

for label in ax5.get_yticklabels():
    label.set_color("white")
    label.set_fontsize(0)

ax6 = fig.add_subplot(2, 3, 3, sharey=ax1)
ax6.semilogy(xticks, cputime, color='black')
ax6.semilogy(xticks, cputime + 1.96 * cputime_std_error, color='grey', linestyle=':')
ax6.semilogy(xticks, cputime - 1.96 * cputime_std_error, color='grey', linestyle=':')
ax6.set_title("n="+str(n))

for label in ax6.get_yticklabels():
    label.set_color("white")
    label.set_fontsize(0)

for label in ax6.get_xticklabels():
    label.set_color("white")
    label.set_fontsize(0)
    
for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
    ax.grid(axis='y')

plt.tight_layout()
plt.savefig("figs/runtime_in_m.pdf")
plt.show()

In [ ]:
plt.plot(df['cputime'].values)
plt.show()

In [ ]:
# # plot runtime in m for n fixed

# df = pd.read_csv("figs/optgap_in_iter_m3n15.csv")
# x = df['iteration']
# opt_gap = df['opt_gap']
# opt_gap_std_error = df['opt_gap_std_error']
# cube_counts = df['cube_counts']
# cube_counts_std_error = df['cube_counts_std_error']
# n = 15
# m = 3

# fig = plt.figure(figsize=(5,5))

# ax1 = fig.add_subplot(2, 1, 1)
# ax1.plot(x, opt_gap, color='black')
# ax1.plot(x, opt_gap + 1.96 * opt_gap_std_error, color='grey', linestyle=':')
# ax1.plot(x, opt_gap - 1.96 * opt_gap_std_error, color='grey', linestyle=':')
# ax1.set_ylabel('optimality gap')

# for label in ax1.get_xticklabels():
#     label.set_color("white")

# ax2 = fig.add_subplot(2, 1, 2, sharex=ax1)

# ax2.plot(x, cube_counts, color='black')
# ax2.plot(x, cube_counts + 1.96 * cube_counts_std_error, color='grey', linestyle=':')
# ax2.plot(x, cube_counts - 1.96 * cube_counts_std_error, color='grey', linestyle=':')

# ax2.set_xticks(np.arange(2,11))
# ax2.set_xlim((1.5,10.5))
# ax2.set_yticks([0,500,1000,1500,2000])
# ax2.set_ylim((-100,2100))
# ax2.set_ylabel('# hypercubes')
# ax2.set_xlabel('# iterations')

# plt.tight_layout()
# plt.savefig("figs/optgap_in_iter_m3n15.pdf")
# plt.show()

In [ ]:
def plot_grid(ax, cubes, cubes_to_plot, segments, x_axis=True, y_axis=True):
    
    c1 = segments[0]
    c2 = segments[1]
    
    _ = np.min([c1.x_lb, c2.x_lb])
    ax.scatter(_, _, facecolor='none')
    _ = np.max([c1.x_ub, c2.x_ub])
    ax.scatter(_, _, facecolor='none')  

    for cube in cubes_to_plot:
        
        if cube.outside_X:
            ax.add_patch(
                patches.Rectangle(
                    cube.bottomcorner,
                    2 * cube.r,
                    2 * cube.r,
                    facecolor='darkgrey',
                    edgecolor='white'
                )
            )
        
        elif not cube.feasible: 
            ax.add_patch(
                patches.Rectangle(
                    cube.bottomcorner,
                    2 * cube.r,
                    2 * cube.r,
                    facecolor='darkgrey',
                    edgecolor='white'
                )
            )
            
        elif cube.suboptimal:
            ax.add_patch(
                patches.Rectangle(
                    cube.bottomcorner,
                    2 * cube.r,
                    2 * cube.r,
                    edgecolor='white',
                    facecolor='salmon',
                )
            )
        
        else:
            raise Exception('Cube is inside X and feasible.')
    
    for i,cube in enumerate(cubes):

        ax.add_patch(
            patches.Rectangle(
                cube.bottomcorner,
                2 * cube.r,
                2 * cube.r,
                facecolor='seagreen',
                edgecolor='white'
            )
        )
        
    ax.add_patch(
        patches.Rectangle(
            (c1.x_lb, c2.x_lb),
            c1.x_ub - c1.x_lb,
            c2.x_ub - c2.x_lb,
            fill=None,
            color='black',
            linestyle='--'
        )
    )
    
    if not x_axis: ax.xaxis.set_ticklabels([])
    if not y_axis: ax.yaxis.set_ticklabels([])
        
no_x_axis = [1,2,3]
no_y_axis = [2,3,5,6]
fig = plt.figure()
for k in range(np.min([6,len(bnb.xlog['cubes'])])):
    
    ax = fig.add_subplot(2, 3, k+1, aspect='equal')
    plot_grid(ax,
              bnb.xlog['cubes'][k],
              bnb.xlog['cubes_to_plot'][k],
              bnb.segments,
              x_axis = False if k+1 in no_x_axis else True,
              y_axis = False if k+1 in no_y_axis else True
             )
    
    ax.set_yticks(ax.get_xticks())
    ax.set_ylim(ax.get_xlim())
    
plt.tight_layout()
plt.show()
# plt.savefig(f'gridfigs/X_all.pdf', bbox_inches='tight')
# plt.close()